In [1]:
from keras.models import load_model
from collections import deque
import numpy as np
import pickle
import cv2

In [2]:
model = load_model('outputmodel.h5')
lb = pickle.loads(open(r'v_c_binarizer.pickle','rb').read())
outputVideo = r'demo_oupt.avi'
mean = np.array([123.68, 116.779, 103.939][::1], dtype='float32')
Queue = deque(maxlen=128)

In [3]:
capturevideo = cv2.VideoCapture('demo.mp4')
writer = None
(width, height) = (None, None)

while True:
    (taken, frame) = capturevideo.read()
    
    if not taken:
        break
    if width is None or height is None:
        (width, height) = frame.shape[:2]
        
    output = frame.copy()
    frame = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
    frame = cv2.resize(frame,(224,224)).astype('float32')
    frame -= mean
    preds = model.predict(np.expand_dims(frame,axis=0))[0]
    Queue.append(preds)
    results = np.array(Queue).mean(axis=0)
    i = np.argmax(results)
    label = lb.classes_[i]
    text = "p: {}".format(label)
    cv2.putText(output,text,(45,60),cv2.FONT_HERSHEY_SIMPLEX, 1.25,(255,0,0), 5)
    
    if writer is None:
        fourcc = cv2.VideoWriter_fourcc(*"MJPG")
        writer = cv2.VideoWriter('outputVideo', fourcc,10,(width, height), True)
    writer.write(output)
    cv2.imshow('In progress',output)
    key = cv2.waitKey(1) & 0xFF
    
    if key == ord('q'):
        break

print('Finalizing.....')
writer.release()
capturevideo.release()

Finalizing.....
